In [178]:
import openrouteservice as ors
import folium

client = ors.Client(key='5b3ce3597851110001cf62480413a152b510459cabf5e54bcfb4a7a9')
#client = ors.Client(key='5b3ce3597851110001cf6248f4b857eea0554d93886b4c09e748b2bf')

In [149]:
import re
import pandas as pd
import math

import numpy as np

from operator import itemgetter
from heapq import nsmallest

from itertools import chain

import time

In [212]:
df_property = pd.read_csv('../data/curated/property.csv')

property_coor = []
for index, row in df_property.iterrows():
    if re.search(r'\[(-\d+.\d+)',str(row['coordinates'])): 
        lat = float(re.findall(r'\[(-\d+.\d+)',str(row['coordinates']))[0])
        long = float(re.findall(r'(\d+.\d+)\]',str(row['coordinates']))[0])
        property_coor.append((long,lat))
    else:
        # coordinate information is missing
        # append the coordinate of New York, USA. It is very far from any points in Australia
        property_coor.append((-74.0060152, 40.7127281))
print(len(property_coor))

df_school = pd.read_csv('../data/curated/school.csv')

school_coor = []
for index, row in df_school.iterrows():
    if not math.isnan(row['Y']):
        lat = row['Y']
        long = row['X']
        school_coor.append((long,lat))
    else:
        school_coor.append((-74.0060152, 40.7127281))
print(len(school_coor))


15226
2299


In [203]:
import geopy.distance

In [201]:
# find the closest 3 schools for each property based on euclidean distance
# api has accessed limits so that it is difficult to track the route between each school and each property


def get_school(index):

    # calculate distance between the property and each school
    distances = [geopy.distance.geodesic((property_coor[index][1], property_coor[index][0]), (school_coor[j][1], school_coor[j][0])).km for j in range(len(school_coor))]

    # find the 3 nearest school
    index = list(range(len(school_coor)))
    pairs = zip(index, distances)
    result = nsmallest(3, pairs, key=itemgetter(1))
    index_list = [i[0] for i in result]

    # store the name and the coordinates of the 3 nearest schools
    name_list = [df_school['School_Name'][i] for i in index_list]
    coor_list = [school_coor[i] for i in index_list]
    
    result = []
    result.append(name_list)
    result.append(coor_list)

    return result

# for each property, get corresponding closest schools
name_list = []
coor_list = []

for i in range(len(property_coor)):
    result = get_school(i)
    name_list.append(result[0])
    coor_list.append(result[1])

print(len(name_list))


15226


In [207]:
print(name_list[1175])

['St Ignatius College Geelong', 'Youth2Industry College', 'Mallacoota P-12 College']


In [204]:
iter = 34
print(coor_list[iter*34:iter*34+34])

[[(144.95615, -37.81654), (144.95442, -37.81289), (144.9535, -37.82935)], [(144.94055, -37.81271), (144.95442, -37.81289), (144.95615, -37.81654)], [(144.94055, -37.81271), (144.95442, -37.81289), (144.95615, -37.81654)], [(144.94055, -37.81271), (144.95615, -37.81654), (144.95442, -37.81289)], [(144.94055, -37.81271), (144.94974, -37.8306), (144.95615, -37.81654)], [(144.94055, -37.81271), (144.95615, -37.81654), (144.94974, -37.8306)], [(144.94055, -37.81271), (144.95442, -37.81289), (144.95288, -37.80597)], [(144.94055, -37.81271), (144.95442, -37.81289), (144.95615, -37.81654)], [(144.95615, -37.81654), (144.95442, -37.81289), (144.94055, -37.81271)], [(144.94055, -37.81271), (144.95442, -37.81289), (144.95288, -37.80597)], [(144.95615, -37.81654), (144.95442, -37.81289), (144.9535, -37.82935)], [(144.94055, -37.81271), (144.95442, -37.81289), (144.95615, -37.81654)], [(144.90684, -37.80116), (144.89376, -37.79104), (144.89267, -37.79838)], [(144.90025, -37.8064), (144.89267, -37.7

In [202]:

def get_school_ors(iter):


    flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
    combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

    if iter == len(df_property)//34:
        count = len(df_property)-iter*34
    else:
        count = 34

    matrix = client.distance_matrix(
        locations=combine_coor,
        profile='driving-car',
        metrics=['distance', 'duration'],
        validate=False,
        destinations=list(range(count,len(combine_coor))),
        sources=list(range(count)),
    )

    duration_list = []
    distance_list = []
    school_name_list = []
    school_coor_list = []


    for i in range(count):

        min_index = np.argmin(matrix['durations'][i][i*3:i*3+3])
        min_index_flat = min_index+i*3
        duration_list.append(matrix['durations'][i][min_index_flat])
        distance_list.append(matrix['distances'][i][min_index_flat])

        school_name_list.append(name_list[iter*34+i][min_index])
        school_coor_list.append(coor_list[iter*34+i][min_index])


    #print("Durations in secs: {}\n".format(matrix['durations']))
    #print("Distances in m: {}".format(matrix['distances']))

    # we only want to call the whole function once at each main iteration
    result_list = []
    result_list.append(duration_list)
    result_list.append(distance_list)
    result_list.append(school_name_list)
    result_list.append(school_coor_list)

    return result_list

# A total of 3500 routes is allowed for each call of api, so that we can pass 34 groups of property 
# with corresponding 3 closest schools at each time

duration_list = []
distance_list = []
school_name_list = []
school_coor_list = []

num_call = len(df_property)//34 + 1
for i in range(34,35):
    result = get_school_ors(i)
    '''
    if i%30 == 0:
        time.sleep(60)
    '''
    duration_list.append(result[0])
    distance_list.append(result[1])
    school_name_list.append(result[2])
    school_coor_list.append(result[3])

print(duration_list)
print(distance_list)
print(school_name_list)
print(school_coor_list)

InvalidJSONError: Out of range float values are not JSON compliant

In [189]:
import requests

def get_school_ors_request(iter):


    flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
    combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

    if iter == len(df_property)//34:
        count = len(df_property)-iter*34
    else:
        count = 34

    body = {"locations":combine_coor,"destinations":list(range(count,len(combine_coor))),"metrics":["distance","duration"],"sources":list(range(count))}

    headers = {
        'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
        'Authorization': '5b3ce3597851110001cf62480413a152b510459cabf5e54bcfb4a7a9',
        'Content-Type': 'application/json; charset=utf-8'
    }
    call = requests.post('https://api.openrouteservice.org/v2/matrix/driving-car', json=body, headers=headers)

    print(call.status_code, call.reason)
    print(iter)
    #print(call.text)

num_call = len(df_property)//34 + 1
for i in range(34,35):
    result = get_school_ors_request(i)
    
    if i%30 == 0:
        time.sleep(60)

InvalidJSONError: Out of range float values are not JSON compliant

In [190]:
iter = 34
print(property_coor[iter*34:iter*34+34])
flatten_list = list(chain.from_iterable(coor_list[iter*34:iter*34+34]))
combine_coor = property_coor[iter*34:iter*34+34] + flatten_list

[(144.9532525, -37.8209691), (144.9400507, -37.8148335), (144.9382439, -37.8156528), (144.943274, -37.8198146), (144.9419443, -37.8211627), (144.943907, -37.8208778), (144.9429762, -37.8132032), (144.9469926, -37.8145933), (144.9480308, -37.8190506), (144.9365953, -37.8129453), (144.9480769, -37.8207394), (144.9395017, -37.8143667), (144.9060052, -37.7958415), (144.8952361, -37.8041246), (144.9009967, -37.806049), (144.8910235, -37.8031645), (144.8888312, -37.7997551), (144.8934737, -37.8042177), (144.8858224, -37.7966876), (0, 0), (144.8869829, -37.8047434), (144.8926999, -37.804392), (144.9065856, -37.8048137), (144.8938605, -37.8061368), (144.888165, -37.805818), (144.8898521, -37.7946395), (144.8922271, -37.8054713), (144.8922271, -37.8054713), (144.8916682, -37.8091111), (144.8904647, -37.79385389999999), (144.8892181, -37.8094805), (144.8893041, -37.7942249), (144.8898414, -37.8066105), (144.9009998, -37.7975694)]


In [144]:
flatten_duration_list = list(chain.from_iterable(duration_list))
flatten_distance_list = list(chain.from_iterable(distance_list))
flatten_school_name_list = list(chain.from_iterable(school_name_list))
flatten_school_coor_list = list(chain.from_iterable(school_coor_list))

print(len(flatten_distance_list))

680
